# CNN model - DVA263 project

# Libraries

Declaring all libraries in use

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L2

# Load Dataset

# Prepare Data

# Build Model

# Compile and Train Model

# Putting everything together

Main section putting each part together and running the classification task